In [1]:
import pandas as pd
import numpy as np
data_aligned_b1 = pd.read_excel('data_aligned_b1.xlsx',
     engine='openpyxl', index_col=0)
data_aligned_b1

,ДатаПробыКокса,M25,M10,CSR,ДатаШихтовки,% спек,% марка Ж,% кокс.,"Оборот печей, ч","Помол, %",...,Vt,I,Io,SI,КТЦ эксп.,MF,MF_spec,MF_otosh,CSR_carb,"Оборот печей отсеч, ч"
0,2017-01-05,88.7,7.7,NaN,2017-01-01,50.0,29.8,28,19.50,77.9,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
1,2017-01-06,88.7,7.5,NaN,2017-01-02,50.0,29.8,28,19.50,77.0,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
2,2017-01-07,88.6,7.6,NaN,2017-01-03,50.0,29.8,28,19.50,76.6,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
3,2017-01-08,88.1,7.6,NaN,2017-01-04,50.0,29.8,28,19.50,76.1,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
4,2017-01-09,88.5,7.8,60.7,2017-01-05,50.0,29.8,28,19.50,77.6,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,2021-10-15,87.3,7.8,60.5,2021-10-11,46.0,35.4,50,18.00,74.8,...,41.39000,20.040000,0.14960,3.785,0.5964,5402.95,5250.0,152.95,40.782,18.0
1708,2021-10-16,87.1,7.8,NaN,2021-10-12,46.0,35.4,50,20.25,74.1,...,41.39000,20.040000,0.14960,3.785,0.5964,5402.95,5250.0,152.95,40.782,18.0
1709,2021-10-17,87.4,8.0,NaN,2021-10-13,46.0,35.4,50,18.00,75.0,...,41.39000,20.040000,0.14960,3.785,0.5964,5402.95,5250.0,152.95,40.782,18.0
1710,2021-10-18,87.2,7.6,NaN,2021-10-14,46.0,35.4,50,20.75,75.4,...,44.07000,21.080000,0.15976,3.585,0.6208,5364.75,5250.0,114.75,44.298,18.0


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [3]:
results = pd.DataFrame(columns=['model', 'description', 'R2', 'MAE', 'MAPE', 'MSE', 'RMSE', 'Accuracy'])

def evaluate(model, test_features, test_labels):
    
    predictions = model.predict(test_features)
    
    R2 = model.score(test_features, test_labels)
    MAE = metrics.mean_absolute_error(test_labels, predictions)
    MAPE = np.mean(np.abs((test_labels - predictions) / np.abs(test_labels)))
    MSE = metrics.mean_squared_error(test_labels, predictions)
    RMSE = np.sqrt(metrics.mean_squared_error(test_labels, predictions))
    
    feature_importances = pd.DataFrame({'features':list(test_features.columns), 
                                       'feature_importances':  model.feature_importances_})\
                        .sort_values(by='feature_importances', ascending=False)
    
    print('Model Perfomance')
    print('Test R2: {:0.4f}'.format(R2))
    print('MA Error: {:0.4f}'.format(MAE))
    print('MAP Error: {:0.4f}'.format(MAPE))
    print('MS Error: {:0.4f}'.format(MSE))
    print('RMS Error: {:0.4f}'.format(RMSE))
    print('Accuracy:', round(100*(1 - MAPE), 4))
    print(feature_importances)
    results.loc[len(results.index)] = [model, '', R2, MAE, MAPE, MSE, RMSE, round(100*(1 - MAPE), 4)]    

я собираюсь каждую модель построить с таргетом м10 отдельно, таргет м25 отдельно.
каждую из них прогнать с выборками трейн/тест 80 на 20 и 70 на 30, 
с бутстрэпом и без
с показателем печей обычным и отредактированным

каждую через gridsearch и по каждой сравнить показатели
вроде, ниче не забыл?

In [4]:
#M25 - target, 22 cols, 20% test
X_1 = data_aligned_b1.iloc[:,5:27]
y_1 = data_aligned_b1['M25']
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.2, random_state=0)

In [59]:
rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_1_train, y_1_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 140}

In [5]:
#model_1
model_1 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_1.fit(X_1_test, y_1_test)

evaluate(model_1, X_1_test, y_1_test)

Model Perfomance
Test R2: 0.8502
MA Error: 0.1090
MAP Error: 0.0012
MS Error: 0.0225
RMS Error: 0.1499
Accuracy: 99.8779
           features  feature_importances
11               R0             0.165060
12                σ             0.115566
5           Пыль, %             0.072189
13               Vt             0.057162
15               Io             0.053332
9       Мд. Серы, %             0.053202
1         % марка Ж             0.045437
16               SI             0.043548
14                I             0.038376
20         MF_otosh             0.037050
4          Помол, %             0.034848
6          Влага, %             0.030099
7           Зола, %             0.028530
8         Летуч., %             0.028408
17        КТЦ эксп.             0.026808
21         CSR_carb             0.026637
3   Оборот печей, ч             0.026224
0            % спек             0.025957
19          MF_spec             0.025513
2           % кокс.             0.025142
18               M

In [6]:
#model_1 with reduced 6 weak features
X_1_rf = X_1.drop(['Пласт. слой, мм', 'КТЦ эксп.', 'Влага, %', 'Зола, %', 'MF_spec', '% кокс.'], axis=1)

X_1_rf_train, X_1_rf_test, y_1_rf_train, y_1_rf_test = train_test_split(X_1_rf, y_1, test_size=0.2, random_state=0)

model_1_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_1_rf.fit(X_1_rf_test, y_1_rf_test)

evaluate(model_1_rf, X_1_rf_test, y_1_rf_test)

Model Perfomance
Test R2: 0.8481
MA Error: 0.1097
MAP Error: 0.0012
MS Error: 0.0228
RMS Error: 0.1509
Accuracy: 99.8771
           features  feature_importances
7                R0             0.177257
8                 σ             0.127057
4           Пыль, %             0.078581
9                Vt             0.069260
11               Io             0.065894
6       Мд. Серы, %             0.062286
12               SI             0.055507
1         % марка Ж             0.053629
13               MF             0.046818
10                I             0.045336
14         MF_otosh             0.043727
3          Помол, %             0.042063
5         Летуч., %             0.035222
15         CSR_carb             0.034904
0            % спек             0.031999
2   Оборот печей, ч             0.030460


In [7]:
#model_1 with reduced 12 weak features
X_1_rf_2 = X_1_rf.drop(['% спек', 'CSR_carb', 'SI', 'Помол, %', 'Летуч., %', 'R0'], axis=1)

X_1_rf_2_train, X_1_rf_2_test, y_1_rf_2_train, y_1_rf_2_test = train_test_split(X_1_rf_2, y_1, test_size=0.2, random_state=0)

model_1_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_1_rf_2.fit(X_1_rf_2_test, y_1_rf_2_test)

evaluate(model_1_rf_2, X_1_rf_2_test, y_1_rf_2_test)

Model Perfomance
Test R2: 0.8278
MA Error: 0.1161
MAP Error: 0.0013
MS Error: 0.0258
RMS Error: 0.1606
Accuracy: 99.87
          features  feature_importances
4                σ             0.240368
7               Io             0.108074
2          Пыль, %             0.108074
5               Vt             0.103928
9         MF_otosh             0.084810
3      Мд. Серы, %             0.079655
0        % марка Ж             0.077139
8               MF             0.075212
6                I             0.067607
1  Оборот печей, ч             0.055134


In [9]:
#M25 - target, 22 cols, 20% test
X_2 = data_aligned_b1.iloc[:,5:28].drop(['Оборот печей, ч'], axis=1)
y_2 = data_aligned_b1['M25']
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.2, random_state=0)

In [68]:

rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_2_train, y_2_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 110}

In [10]:
#model_2
model_2 = RandomForestRegressor(n_estimators=110,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_2.fit(X_2_test, y_2_test)

evaluate(model_2, X_2_test, y_2_test)

Model Perfomance
Test R2: 0.8477
MA Error: 0.1097
MAP Error: 0.0012
MS Error: 0.0228
RMS Error: 0.1511
Accuracy: 99.8771
                 features  feature_importances
10                     R0             0.171298
11                      σ             0.123086
4                 Пыль, %             0.069776
12                     Vt             0.057064
14                     Io             0.051857
1               % марка Ж             0.051476
8             Мд. Серы, %             0.051292
15                     SI             0.040358
13                      I             0.040053
19               MF_otosh             0.037962
3                Помол, %             0.033000
5                Влага, %             0.031119
6                 Зола, %             0.030550
7               Летуч., %             0.028724
2                 % кокс.             0.027717
16              КТЦ эксп.             0.027292
17                     MF             0.027076
20               CSR_carb        

In [11]:
#model_2 with reduced 6 weak features
X_2_rf = X_2.drop(['Пласт. слой, мм', 'Оборот печей отсеч, ч', 'КТЦ эксп.', 'Влага, %', 'Зола, %', 'MF_spec'], axis=1)

X_2_rf_train, X_2_rf_test, y_2_rf_train, y_2_rf_test = train_test_split(X_2_rf, y_2, test_size=0.2, random_state=0)

model_2_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_2_rf.fit(X_2_rf_test, y_2_rf_test)

evaluate(model_2_rf, X_2_rf_test, y_2_rf_test)

Model Perfomance
Test R2: 0.8492
MA Error: 0.1097
MAP Error: 0.0012
MS Error: 0.0226
RMS Error: 0.1504
Accuracy: 99.8771
       features  feature_importances
7            R0             0.180954
8             σ             0.127003
4       Пыль, %             0.077508
9            Vt             0.066797
11           Io             0.066468
6   Мд. Серы, %             0.058065
1     % марка Ж             0.055781
12           SI             0.055551
13           MF             0.047454
10            I             0.046385
14     MF_otosh             0.044031
3      Помол, %             0.042286
5     Летуч., %             0.035504
15     CSR_carb             0.034687
0        % спек             0.031132
2       % кокс.             0.030395


In [12]:
#model_2 with reduced 12 weak features
X_2_rf_2 = X_2_rf.drop(['% кокс.', 'SI', '% спек', 'CSR_carb', 'Помол, %', 'Летуч., %'], axis=1)

X_2_rf_2_train, X_2_rf_2_test, y_2_rf_2_train, y_2_rf_2_test = train_test_split(X_2_rf_2, y_2, test_size=0.2, random_state=0)

model_2_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_2_rf_2.fit(X_2_rf_2_test, y_2_rf_2_test)

evaluate(model_2_rf_2, X_2_rf_2_test, y_2_rf_2_test)

Model Perfomance
Test R2: 0.8292
MA Error: 0.1164
MAP Error: 0.0013
MS Error: 0.0256
RMS Error: 0.1600
Accuracy: 99.8696
      features  feature_importances
3           R0             0.204395
4            σ             0.151712
1      Пыль, %             0.096311
5           Vt             0.095217
7           Io             0.090699
2  Мд. Серы, %             0.084179
0    % марка Ж             0.073722
8           MF             0.070268
6            I             0.069269
9     MF_otosh             0.064229


In [13]:
#M10 - target, 22 cols, 20% test
X_3 = data_aligned_b1.iloc[:,5:27]
y_3 = data_aligned_b1['M10']
X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=0.2, random_state=0)

In [74]:
rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_3_train, y_3_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 140}

In [14]:
#model_3
model_3 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=4,
                                min_samples_split=2, 
                                random_state=0)
model_3.fit(X_3_test, y_3_test)

evaluate(model_3, X_3_test, y_3_test)

Model Perfomance
Test R2: 0.7011
MA Error: 0.1037
MAP Error: 0.0146
MS Error: 0.0200
RMS Error: 0.1414
Accuracy: 98.5433
           features  feature_importances
12                σ             0.239733
5           Пыль, %             0.106400
1         % марка Ж             0.054794
15               Io             0.049716
6          Влага, %             0.044845
20         MF_otosh             0.044078
14                I             0.043489
4          Помол, %             0.041785
11               R0             0.038489
13               Vt             0.036637
16               SI             0.032396
7           Зола, %             0.031486
21         CSR_carb             0.029009
18               MF             0.028477
2           % кокс.             0.026366
8         Летуч., %             0.025788
3   Оборот печей, ч             0.024913
0            % спек             0.023947
19          MF_spec             0.022500
9       Мд. Серы, %             0.021308
17        КТЦ эксп

In [15]:
#model_3 with reduced 6 weak features
X_3_rf = X_3.drop(['Пласт. слой, мм', 'MF_spec', 'Оборот печей, ч', 'MF', '% кокс.', '% спек'], axis=1)

X_3_rf_train, X_3_rf_test, y_3_rf_train, y_3_rf_test = train_test_split(X_3_rf, y_3, test_size=0.2, random_state=0)

model_3_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_3_rf.fit(X_3_rf_test, y_3_rf_test)

evaluate(model_3_rf, X_3_rf_test, y_3_rf_test)

Model Perfomance
Test R2: 0.7980
MA Error: 0.0863
MAP Error: 0.0121
MS Error: 0.0135
RMS Error: 0.1163
Accuracy: 98.788
       features  feature_importances
8             σ             0.228750
2       Пыль, %             0.102303
11           Io             0.065395
0     % марка Ж             0.064827
14     MF_otosh             0.060460
3      Влага, %             0.058584
10            I             0.052714
1      Помол, %             0.051046
4       Зола, %             0.050611
12           SI             0.048291
7            R0             0.045489
5     Летуч., %             0.039246
9            Vt             0.037942
6   Мд. Серы, %             0.034704
15     CSR_carb             0.034276
13    КТЦ эксп.             0.025361


In [16]:
#model_3 with reduced 12 weak features
X_3_rf_2 = X_3_rf.drop(['Vt', 'SI', 'КТЦ эксп.', 'Зола, %', 'CSR_carb', 'R0'], axis=1)

X_3_rf_2_train, X_3_rf_2_test, y_3_rf_2_train, y_3_rf_2_test = train_test_split(X_3_rf_2, y_3, test_size=0.2, random_state=0)

model_3_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_3_rf_2.fit(X_3_rf_2_test, y_3_rf_2_test)

evaluate(model_3_rf_2, X_3_rf_2_test, y_3_rf_2_test)

Model Perfomance
Test R2: 0.7931
MA Error: 0.0879
MAP Error: 0.0123
MS Error: 0.0138
RMS Error: 0.1177
Accuracy: 98.7653
      features  feature_importances
6            σ             0.277623
2      Пыль, %             0.121594
8           Io             0.100886
0    % марка Ж             0.085475
9     MF_otosh             0.082190
7            I             0.080947
3     Влага, %             0.075526
1     Помол, %             0.068648
5  Мд. Серы, %             0.054815
4    Летуч., %             0.052297


In [17]:
#M10 - target, 22 cols, 20% test
X_4 = data_aligned_b1.iloc[:,5:28].drop(['Оборот печей, ч'], axis=1)
y_4 = data_aligned_b1['M10']
X_4_train, X_4_test, y_4_train, y_4_test = train_test_split(X_4, y_4, test_size=0.2, random_state=0)

In [79]:
rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_4_train, y_4_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 140}

In [18]:
#model_4
model_4 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=4,
                                min_samples_split=2, 
                                random_state=0)
model_4.fit(X_4_test, y_4_test)

evaluate(model_4, X_4_test, y_4_test)

Model Perfomance
Test R2: 0.6990
MA Error: 0.1040
MAP Error: 0.0146
MS Error: 0.0201
RMS Error: 0.1419
Accuracy: 98.5397
                 features  feature_importances
11                      σ             0.241109
4                 Пыль, %             0.107685
1               % марка Ж             0.053662
14                     Io             0.049835
5                Влага, %             0.045991
13                      I             0.045713
19               MF_otosh             0.044895
3                Помол, %             0.042059
10                     R0             0.040110
12                     Vt             0.036201
15                     SI             0.032657
6                 Зола, %             0.031699
17                     MF             0.030782
20               CSR_carb             0.028355
2                 % кокс.             0.027971
7               Летуч., %             0.026138
0                  % спек             0.025499
8             Мд. Серы, %        

In [19]:
#model_4 with reduced 6 weak features
X_4_rf = X_4.drop(['Пласт. слой, мм', 'Оборот печей отсеч, ч', 'MF_spec', 'MF', '% кокс.', '% спек'], axis=1)

X_4_rf_train, X_4_rf_test, y_4_rf_train, y_4_rf_test = train_test_split(X_4_rf, y_4, test_size=0.2, random_state=0)

model_4_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_4_rf.fit(X_4_rf_test, y_4_rf_test)

evaluate(model_4_rf, X_4_rf_test, y_4_rf_test)

Model Perfomance
Test R2: 0.7980
MA Error: 0.0863
MAP Error: 0.0121
MS Error: 0.0135
RMS Error: 0.1163
Accuracy: 98.788
       features  feature_importances
8             σ             0.228750
2       Пыль, %             0.102303
11           Io             0.065395
0     % марка Ж             0.064827
14     MF_otosh             0.060460
3      Влага, %             0.058584
10            I             0.052714
1      Помол, %             0.051046
4       Зола, %             0.050611
12           SI             0.048291
7            R0             0.045489
5     Летуч., %             0.039246
9            Vt             0.037942
6   Мд. Серы, %             0.034704
15     CSR_carb             0.034276
13    КТЦ эксп.             0.025361


In [20]:
#model_4 with reduced 12 weak features
X_4_rf_2 = X_4_rf.drop(['Vt', 'SI', 'КТЦ эксп.', 'Зола, %', 'CSR_carb', 'R0'], axis=1)

X_4_rf_2_train, X_4_rf_2_test, y_4_rf_2_train, y_4_rf_2_test = train_test_split(X_4_rf_2, y_4, test_size=0.2, random_state=0)

model_4_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_4_rf_2.fit(X_4_rf_2_test, y_4_rf_2_test)

evaluate(model_4_rf_2, X_4_rf_2_test, y_4_rf_2_test)

Model Perfomance
Test R2: 0.7931
MA Error: 0.0879
MAP Error: 0.0123
MS Error: 0.0138
RMS Error: 0.1177
Accuracy: 98.7653
      features  feature_importances
6            σ             0.277623
2      Пыль, %             0.121594
8           Io             0.100886
0    % марка Ж             0.085475
9     MF_otosh             0.082190
7            I             0.080947
3     Влага, %             0.075526
1     Помол, %             0.068648
5  Мд. Серы, %             0.054815
4    Летуч., %             0.052297


In [21]:
results.description = ['M25_22_Оборот печей, ч', 'M25_16_Оборот печей, ч', 'M25_10_Оборот печей, ч', 
                       'M25_22_Оборот печей отсеч, ч', 'M25_16_Оборот печей отсеч, ч', 'M25_10_Оборот печей отсеч, ч',
                       'M10_22_Оборот печей, ч', 'M10_16_Оборот печей, ч', 'M10_10_Оборот печей, ч', 
                       'M10_22_Оборот печей отсеч, ч', 'M10_16_Оборот печей отсеч, ч', 'M10_10_Оборот печей отсеч, ч']
results

,model,description,R2,MAE,MAPE,MSE,RMSE,Accuracy
0,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_22_Оборот печей, ч",0.850188,0.109015,0.001221,0.022457,0.149858,99.8779
1,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_16_Оборот печей, ч",0.848128,0.109704,0.001229,0.022766,0.150884,99.8771
2,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_10_Оборот печей, ч",0.827846,0.116081,0.001300,0.025806,0.160644,99.8700
3,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_22_Оборот печей отсеч, ч",0.847670,0.109701,0.001229,0.022835,0.151112,99.8771
4,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_16_Оборот печей отсеч, ч",0.849187,0.109739,0.001229,0.022607,0.150357,99.8771
5,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_10_Оборот печей отсеч, ч",0.829178,0.116418,0.001304,0.025607,0.160021,99.8696
6,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_22_Оборот печей, ч",0.701111,0.103699,0.014567,0.019999,0.141418,98.5433
7,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_16_Оборот печей, ч",0.797960,0.086311,0.012120,0.013519,0.116270,98.7880
8,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_10_Оборот печей, ч",0.793051,0.087927,0.012347,0.013847,0.117674,98.7653
9,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_22_Оборот печей отсеч, ч",0.699045,0.103954,0.014603,0.020137,0.141906,98.5397
